In [1]:
using JuMP, CSV, DataFrames, Gurobi, Interpolations

In [2]:
# Read the test and train CSV files
data_targeted = CSV.File("/Users/sanyachauhan/Downloads/Data_Targeted.csv") |> DataFrame;
data_email = CSV.File("/Users/sanyachauhan/Downloads/Data_Emails.csv") |> DataFrame;
data_print = CSV.File("/Users/sanyachauhan/Downloads/Data_Print.csv") |> DataFrame;
data_influencer = CSV.File("/Users/sanyachauhan/Downloads/Data_Influencers.csv") |> DataFrame;

In [3]:
# Budget levels
B = [1000.0, 1500.0, 2000.0];

In [4]:
# Creating linear interpolation functions for each marketing option
interp_targeted = LinearInterpolation(data_targeted[!, :Cost], data_targeted[!, :Buys]);
interp_email = LinearInterpolation(data_email[!, :Cost], data_email[!, :Buys]);
interp_print = LinearInterpolation(data_print[!, :Cost], data_print[!, :Buys]);
interp_influencers = LinearInterpolation(data_influencer[!, :Cost], data_influencer[!, :Buys]);

In [5]:
for Budget in B
    # Optimization model
    model = Model(Gurobi.Optimizer)
    
    # Decision variables for investments
    @variable(model, investment_targeted >= 0)
    @variable(model, investment_email >= 0)
    @variable(model, investment_print >= 0)
    @variable(model, investment_influencers >= 0)
    
    # Example for Targeted Online Advertising
    @variable(model, targeted_segments[1:3] >= 0)
    breakpoints_targeted = [0, 40, 310, 990] # Example breakpoints
    slopes_targeted = [3, 4, 5] # Example slopes
    @constraint(model, investment_targeted == sum(targeted_segments))
    @constraint(model, sum(targeted_segments[i] for i in 1:3) <= breakpoints_targeted[end])
    for i in 1:3
        @constraint(model, targeted_segments[i] <= breakpoints_targeted[i+1] - breakpoints_targeted[i])
    end
    
    # Example for Email Marketing
    @variable(model, email_segments[1:3] >= 0)
    breakpoints_email = [0, 50, 200, 500] # Example breakpoints
    slopes_email = [2, 3, 4] # Example slopes
    @constraint(model, investment_email == sum(email_segments))
    @constraint(model, sum(email_segments[i] for i in 1:3) <= breakpoints_email[end])
    for i in 1:3
        @constraint(model, email_segments[i] <= breakpoints_email[i+1] - breakpoints_email[i])
    end
    
    # Example for Print Media
    @variable(model, print_segments[1:3] >= 0)
    breakpoints_print = [0, 100, 300, 700] # Example breakpoints
    slopes_print = [1, 2, 3] # Example slopes
    @constraint(model, investment_print == sum(print_segments))
    @constraint(model, sum(print_segments[i] for i in 1:3) <= breakpoints_print[end])
    for i in 1:3
        @constraint(model, print_segments[i] <= breakpoints_print[i+1] - breakpoints_print[i])
    end
    
    # Example for Influencer Marketing
    @variable(model, influencers_segments[1:3] >= 0)
    breakpoints_influencers = [0, 100, 250, 600] # Example breakpoints
    slopes_influencers = [5, 6, 7] # Example slopes
    @constraint(model, investment_influencers == sum(influencers_segments))
    @constraint(model, sum(influencers_segments[i] for i in 1:3) <= breakpoints_influencers[end])
    for i in 1:3
        @constraint(model, influencers_segments[i] <= breakpoints_influencers[i+1] - breakpoints_influencers[i])
    end


    # Objective: Maximize buys
    @objective(model, Max, sum(slopes_targeted[i] * targeted_segments[i] for i in eachindex(slopes_targeted)) + 
                  sum(slopes_email[i] * email_segments[i] for i in eachindex(slopes_email)) +
                  sum(slopes_print[i] * print_segments[i] for i in eachindex(slopes_print)) +
                  sum(slopes_influencers[i] * influencers_segments[i] for i in eachindex(slopes_influencers)))

    # Budget constraint
    @constraint(model, Budget, investment_targeted + investment_email +
                       investment_print + investment_influencers <= Budget)

    # Solve the model
    optimize!(model)

    # Save the model in LP format with a dynamic name
    model_filename = "model2_" * string(Budget) * ".lp"
    write_to_file(model, model_filename)

    # Results
    println("Optimal Investments with Budget = ", Budget)
    println("Targeted Online Advertising: \$", value(investment_targeted))
    println("Email Marketing: \$", value(investment_email))
    println("Print Media: \$", value(investment_print))
    println("Influencer Marketing: \$", value(investment_influencers))
    println("Total Buys: ", objective_value(model))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-31
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 16 columns and 44 nonzeros
Model fingerprint: 0x36fcccdc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+03]
Presolve removed 20 rows and 9 columns
Presolve time: 0.00s
Presolved: 1 rows, 7 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+03   8.125000e+01   0.000000e+00      0s
       1    5.8500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.850000000e+03

User-callback calls 67, time in user-callback 0.00 sec
Optimal Investments with Budget = Budget : investment_tar